# OSMnx + igraph for faster performance

In [1]:
import operator

import igraph as ig
import networkx as nx
import numpy as np
import osmnx as ox

%matplotlib inline
print(ox.__version__)
print(ig.__version__)

weight = "length"

1.3.0
0.10.4


# Construct graphs

In [2]:
# create networkx graph
G_nx = ox.graph_from_place("Piedmont, CA, USA", network_type="drive")
osmids = list(G_nx.nodes)
G_nx = nx.relabel.convert_node_labels_to_integers(G_nx)

# give each node its original osmid as attribute since we relabeled them
osmid_values = {k: v for k, v in zip(G_nx.nodes, osmids)}
nx.set_node_attributes(G_nx, osmid_values, "osmid")

In [3]:
%%time
# convert networkx graph to igraph
G_ig = ig.Graph(directed=True)
G_ig.add_vertices(G_nx.nodes)
G_ig.add_edges(G_nx.edges())
G_ig.vs["osmid"] = osmids
G_ig.es[weight] = list(nx.get_edge_attributes(G_nx, weight).values())

CPU times: total: 0 ns
Wall time: 1.99 ms


In [4]:
assert len(G_nx.nodes()) == G_ig.vcount()
assert len(G_nx.edges()) == G_ig.ecount()

# Shortest paths

In [6]:
source = list(G_nx.nodes())[0]
target = list(G_nx.nodes())[-1]

In [7]:
%%time
path1 = G_ig.get_shortest_paths(v=source, to=target, weights=weight)[0]

CPU times: total: 0 ns
Wall time: 0 ns


In [8]:
%%time
path2 = nx.shortest_path(G_nx, source, target, weight=weight)

CPU times: total: 0 ns
Wall time: 0 ns


In [9]:
assert path1 == path2

In [10]:
%%time
path_length1 = G_ig.distances(source=source, target=target, weights=weight)[0][0]

CPU times: total: 0 ns
Wall time: 0 ns


In [11]:
%%time
path_length2 = nx.shortest_path_length(G_nx, source, target, weight)

CPU times: total: 0 ns
Wall time: 0 ns


In [12]:
assert path_length1 == path_length2

# Centrality analysis (merkezilik analizi) 

In [14]:
%%time
closeness1 = G_ig.closeness(vertices=None, mode="ALL", cutoff=None, weights=weight, normalized=True)
max_closeness1 = np.argmax(closeness1)

CPU times: total: 0 ns
Wall time: 9.98 ms


In [15]:
%%time
closeness2 = nx.closeness_centrality(G_nx, distance=weight, wf_improved=True)
max_closeness2 = max(closeness2.items(), key=operator.itemgetter(1))[0]

CPU times: total: 328 ms
Wall time: 333 ms


In [16]:
# confirm same node has max closeness in both graphs
assert G_nx.nodes[max_closeness2]["osmid"] == G_ig.vs[max_closeness1]["osmid"]